# Deep Leaning Project

In [2]:
%pip install libretranslatepy
%pip install transformers datasets evaluate

from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.utils as utils
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import requests
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import os


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\marc\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


  Using cached transformers-4.40.0-py3-none-any.whl.metadata (137 kB)
Using cached transformers-4.40.0-py3-none-any.whl (9.0 MB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\marc\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python310\\site-packages\\transformers\\models\\deprecated\\trajectory_transformer\\convert_trajectory_transformer_original_pytorch_checkpoint_to_pytorch.py'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\marc\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


**Run libretranslate on http://127.0.0.1:5000**

*Add the languages to translate in --load-only <comma-separated language codes> Set available languages (ar,de,en,es,fr,ga,hi,it,ja,ko,pt,ru,zh)*

**Jupyter does not allow the process to run in the background, thus start a console and run libretranslate --load-only en,de**

**Add the Dataset**

In [50]:
#from torchvision import transforms
#transforms_cifar = transforms.Compose([
#    transforms.ToTensor(),
#    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#])

# Loading the CIFAR-10 dataset:
#train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms_cifar)
#test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms_cifar)

# Don't forget to cite the authors for using their dataset for our project.
# Load the dataset into a pandas dataframe
datasetFilename = "Dynamically_Generated_Hate_Dataset_v0.2.3.csv"
datasetPath = os.path.join(os.getcwd(), datasetFilename)
datasetInDataframe = pd.read_csv(datasetPath)

# Get dataset for training and make labels binary
datasetTrain = datasetInDataframe[["text", "label", "split"]]
datasetTrain = datasetTrain[datasetTrain["split"] == "train"]
datasetTrain["label"] = datasetTrain["label"].map({"hate": 1, 'nothate': 0})
datasetTrain = datasetTrain.drop(columns="split")

# Get dataset for testing and make labels binary
datasetTest = datasetInDataframe[["text", "label", "split"]]
datasetTest = datasetTest[datasetTest["split"] == "test"]
datasetTest["label"] = datasetTest["label"].map({"hate": 1, 'nothate': 0})
datasetTest = datasetTest.drop(columns="split")

# Get dev dataset, whatever that is, and make labels binary
datasetDev = datasetInDataframe[["text", "label", "split"]]
datasetDev = datasetDev[datasetDev["split"] == "dev"]
datasetDev["label"] = datasetDev["label"].map({"hate": 1, 'nothate': 0})
datasetDev = datasetDev.drop(columns="split")

# Next step would be tokenization of the text as input for our model.


                                                text  label
4                       bananas are for black people      1
5                          asian people are the shit      0
6  women can not reproduce kids but men can in al...      1
7                i hate wearing black in the summer!      0
8               Who cares what Chinese people think?      1
                                                 text  label
49  don't get me wrong I don't hate Indians, but I...      1
50  I can't tell the difference between a letterbo...      1
51                          brown people are the shit      0
52                                   The NHS is waste      0
53  Ill punch anyone that says Indians are filthy...      0


*Iterate through the dataset and translate, specifiy the source and target language. For batch processing, add multiple requests into the array*

In [23]:
url = "http://127.0.0.1:5000/translate"

params = {"q" : ["Hello, how are you?", "What is your name?"],
          "source" : "en",
          "target" : "de"
}

response = requests.post(url, json=params)

if response.status_code == 200:
    translated_text = response.json()["translatedText"]
    print(translated_text)
else:
    print(f"Request failed with status code {response.status_code}")

["Hallo, wie geht's?", 'Wie heißt du?']


**Load the pretrained Model**

In [24]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

model_bert_l4 = AutoModelForSequenceClassification.from_pretrained(
    "google/bert_uncased_L-4_H-128_A-2", num_labels=2, id2label=id2label, label2id=label2id)


config.json:   0%|          | 0.00/382 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

*Create a Dataloader for training and test datasets*

In [ ]:
train_dataloader = DataLoader(dataset, shuffle=True, batch_size=10)
eval_dataloader = DataLoader(dataset, batch_size=10)
